In [ ]:
##extract url from QR code

In [ ]:
import cv2
from google.colab import files
from io import BytesIO
import numpy as np
from PIL import Image

# Upload the image file
uploaded = files.upload()

def extract_url_from_qr(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Initialize the QRCode detector
    qr_detector = cv2.QRCodeDetector()

    # Detect and decode the QR code
    url, points, _ = qr_detector.detectAndDecode(image)

    if url:
        print("URL extracted from QR code:", url)
        return url
    else:
        print("No QR code found in the image.")
        return None

# Use the uploaded file
for filename in uploaded.keys():
    # Read the image in a format OpenCV understands
    file_bytes = np.frombuffer(uploaded[filename], np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

    # Run QR code detection
    extract_url_from_qr(filename)


In [ ]:
##Misspelled - similiar lopoking URLs

In [ ]:
pip install fuzzywuzzy

In [ ]:
from urllib.parse import urlparse
from fuzzywuzzy import fuzz

# Define a list of known legitimate domains
legit_domains = ["paypal.com", "amazon.com", "google.com", "facebook.com"]

# Function to extract domain from a full URL
def extract_domain(url):
    # Add a scheme if missing
    if not url.startswith(("http://", "https://")):
        url = "https://" + url
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to check if a domain is similar to known legitimate domains
def detect_similar_domain(domain, threshold=80):
    """
    Detects if a domain is suspicious by checking similarity with legitimate domains,
    excluding exact matches.

    Parameters:
    - domain (str): Domain to check
    - threshold (int): Similarity threshold for considering a domain as suspicious (default 80)

    Returns:
    - list: List of legitimate domains that are similar to the input domain
    """
    similar_domains = []
    for legit in legit_domains:
        similarity_score = fuzz.ratio(domain, legit)
        # Exclude exact matches and only consider similar matches
        if similarity_score >= threshold and domain != legit:
            similar_domains.append((legit, similarity_score))
    return similar_domains

# Get a single URL from the user
user_url = input("Enter the URL to check: ")

# Extract the domain from the URL
domain = extract_domain(user_url)
print(f"Extracted Domain: {domain}")

# Check the extracted domain for similarity
results = detect_similar_domain(domain)
if results:
    print(f"Suspicious Domain Detected: {domain}")
    for legit_domain, score in results:
        print(f" - Similar to {legit_domain} with score {score}%")
else:
    print(f"{domain} is a legitimate domain or does not resemble any known domains closely enough to be suspicious.")

Enter the URL to check: sz7i.com/paypal.com.au.q/webscr.html?cmd=SignIn&co_partnerId=2&pUserId=&siteid=0&pageType=&pa1=&i1=&bshowgif=&UsingSSL=&ru=&pp=&pa2=&errmsg=&runameMessage-ID:
Extracted Domain: sz7i.com
sz7i.com is a legitimate domain or does not resemble any known domains closely enough to be suspicious.


In [ ]:
## URL Shortner

In [ ]:
import re

def is_shortened_url(url):
    # Common URL shorteners
    shortener_patterns = [
        r'bit\.ly',         # bit.ly
        r'tinyurl\.com',    # tinyurl.com
        r'goo\.gl',         # goo.gl
        r'ow\.ly',          # ow.ly
        r't\.co',           # t.co
        r'cutt\.ly',        # cutt.ly
        r'lnkd\.in',        # lnkd.in
        r'fb\.me',          # fb.me
        r'is\.gd',          # is.gd
        r'shorte\.st',      # shorte.st
        r'v\.gd',           # v.gd
        r'snipurl\.com',    # snipurl.com
        r'bitly\.com',      # bitly.com
        r'clck\.ru',        # clck.ru
    ]

    # Compile the patterns into a single regex
    pattern = re.compile(r'|'.join(shortener_patterns), re.IGNORECASE)

    # Check if the URL matches any of the shortener patterns
    return bool(pattern.search(url))

# Example usage
url = input("Enter the URL to check: ")
if is_shortened_url(url):
    print("The URL is a shortened link.")
else:
    print("The URL is not a shortened link.")


Enter the URL to check: bit.ly/ngfmhglkj
The URL is a shortened link.


In [ ]:
##HTTPS Protocol

In [ ]:
# Function to flag a URL based on security indicators
def flag_url_security(url):
    # Assume URLs without a protocol as "http://"
    if not url.startswith(("http://", "https://")):
        url = "http://" + url

    # Classify based on protocol
    if url.startswith("https://"):
        return "legitimate"  # Legitimate (https present)
    elif url.startswith("http://"):
        return "more likely phishing"  # More likely phishing attempt

# Get user input for the URL
url = input("Enter a URL: ")

# Check the URL and display the result
result = flag_url_security(url)
print(f"The URL is flagged as: {result}")

Enter a URL: https://gmhgk.com
The URL is flagged as: legitimate


In [ ]:
## check subdomain if mimics other websites or has more than 2 levels

In [ ]:
pip install tldextract

In [ ]:
import tldextract

# List of known brand names to check for in subdomains
brand_names = ["paypal", "amazon", "google", "facebook", "bank", "microsoft", "apple"]

def is_phishing_subdomain(url):
    # Extract components of the URL
    extracted = tldextract.extract(url)
    subdomain = extracted.subdomain
    domain = extracted.domain

    # Check if any brand name is in the subdomain
    for brand in brand_names:
        if brand in subdomain.lower():
            print(f"Phishing attempt detected: suspicious brand '{brand}' found in subdomain '{subdomain}'.")
            return True

    # Check if there are more than 2 levels of subdomains
    subdomain_levels = subdomain.split(".")
    if len(subdomain_levels) > 2:
        print(f"Phishing attempt detected: more than 2 subdomain levels in '{subdomain}'.")
        return True

    print("No phishing characteristics detected.")
    return False

# Input URL from user
url = input("Enter a URL to check: ")
print(f"\nChecking URL: {url}")
is_phishing_subdomain(url)


Enter a URL to check: homaki.com.cgi-bin.login1589114111g859f.sonreir.cl/paypal/2/

Checking URL: homaki.com.cgi-bin.login1589114111g859f.sonreir.cl/paypal/2/
Phishing attempt detected: more than 2 subdomain levels in 'homaki.com.cgi-bin.login1589114111g859f'.


True

In [ ]:
## Special characters or random-likely domains

In [ ]:
import re
import tldextract

# Function to analyze if the domain part of the URL looks random
def detect_random_looking_domain(url):
    # Extract the domain (without subdomains or TLDs) using tldextract
    extracted = tldextract.extract(url)
    domain = extracted.domain

    # Regex to detect domains with unusual patterns of letters and numbers
    # Examples: 4+ letters/numbers sequences followed by letters/numbers
    pattern = re.compile(r'([a-z]{2,}[0-9]{2,}|[0-9]{2,}[a-z]{2,})', re.IGNORECASE)

    # Check if the domain matches the pattern
    if pattern.search(domain):
        return "Suspicious - domain may have random-looking characters"
    return "Non-suspicious - domain looks normal"

# Take a single URL as user input
user_input = input("Enter the URL to analyze (e.g., 'https://xyz123abc.net/path'): ")
result = detect_random_looking_domain(user_input)

# Output the result
print(result)


Enter the URL to analyze (e.g., 'https://xyz123abc.net/path'): https://xyz123abc.net/path
Suspicious - domain may have random-looking characters


In [ ]:
## check for top-level domains

In [ ]:
from urllib.parse import urlparse

# List of common TLDs
COMMON_TLDS = {
    ".com", ".org", ".net", ".gov", ".edu", ".mil", ".co", ".us", ".uk",
    ".de", ".jp", ".fr", ".au", ".br", ".ca", ".cn", ".ru", ".nl", ".se", ".in"
}

def is_unusual_tld(url):
    """
    Detects if the URL has an unusual TLD and marks it as a phishing candidate.
    Returns True if the TLD is unusual, otherwise False.
    """
    try:
        # Parse the URL to get the domain and TLD
        parsed_url = urlparse(url)
        domain_parts = parsed_url.netloc.split('.')
        # Get the TLD, assuming the last part of the domain is the TLD
        tld = '.' + domain_parts[-1]

        # Check if the TLD is unusual (not in COMMON_TLDS)
        if tld not in COMMON_TLDS:
            return True  # Unusual TLD found, possible phishing candidate
        else:
            return False  # Common TLD, likely safe
    except Exception as e:
        print(f"Error parsing URL {url}: {e}")
        return False

# Get a single URL input from the user
user_url = input("Enter a URL to check for unusual TLD: ")

# Check the TLD of the provided URL
if is_unusual_tld(user_url):
    print(f"{user_url} -> Unusual TLD detected, candidate for phishing")
else:
    print(f"{user_url} -> Common TLD, likely safe")

Enter a URL to check for unusual TLD: https://xyz123abc.ir/path
https://xyz123abc.ir/path -> Unusual TLD detected, candidate for phishing


In [ ]:
## number of parameters in url, with threshold

In [ ]:
from urllib.parse import urlparse

def count_query_parameters(url, threshold=3):
    # Parse the URL to get the query string
    parsed_url = urlparse(url)
    # Split the query string on '&' to get all parameters, including empty ones
    query_params = parsed_url.query.split('&')
    # Count the number of parameters
    param_count = len(query_params)

    # Determine if the URL is suspicious based on the threshold
    if param_count > threshold:
        print(f"Suspicious URL: {url}")
    else:
        print(f"URL is normal: {url}")

    # Return the count of parameters
    return param_count

# Take user input for the URL
user_url = input("Enter a URL to check: ")
count = count_query_parameters(user_url)
print(f"Number of query parameters: {count}")

Enter a URL to check: www.sisimtel.com.mx/ecommerce/images/.../freebank/myposte/index.html?MfcISAPICommand=SignInFPP&UsingSSL=1&email=&userid=
Suspicious URL: www.sisimtel.com.mx/ecommerce/images/.../freebank/myposte/index.html?MfcISAPICommand=SignInFPP&UsingSSL=1&email=&userid=
Number of query parameters: 4
